In [1]:
import pickle
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import Dataset
from torch_geometric.data import Data
import numpy as np
from collections import Counter
from sklearn.metrics import confusion_matrix, balanced_accuracy_score, accuracy_score
from torch.utils.data import Dataset, DataLoader
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [2]:
class Customized_Dataset(Dataset):
    def __init__(self, metadata):
        super(Customized_Dataset, self).__init__()
        self.data = pickle.load(open(metadata, 'rb'))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        with torch.no_grad():
            text_embed = self.data[idx]['text_embed']
            audio_embed = self.data[idx]['audio_embed']
            label = self.data[idx]['label']

        return text_embed, audio_embed, label

In [ ]:
from sklearn.cluster import KMeans

BATCH_SIZE = 128
train_metadata = "C:/Users/admin/Documents/Speech-Emotion_Recognition-2/features/IEMOCAP_BERT_wav2vec_train.pkl"
val_metadata = "C:/Users/admin/Documents/Speech-Emotion_Recognition-2/features/IEMOCAP_BERT_wav2vec_val.pkl"
test_metadata = "C:/Users/admin/Documents/Speech-Emotion_Recognition-2/features/IEMOCAP_BERT_wav2vec_test.pkl"
train_dataset = Customized_Dataset(train_metadata)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_dataset = Customized_Dataset(val_metadata)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

c:\Users\admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Cluster centers:
 [[ 0.06929798  0.14106652 -0.08451123 ... -0.16698426  0.26814625
   0.46567813]
 [-0.18202768  0.10089511 -0.07283699 ... -0.23904952  0.33501562
   0.3424734 ]
 [ 0.10259169  0.08463533 -0.13807371 ... -0.24510658  0.25054204
   0.5270821 ]
 [ 0.1950207   0.06847898  0.04582243 ... -0.18436979  0.32310432
   0.5562032 ]]
Labels:
 [3 0 0 ... 3 0 2]


In [4]:
class MMSER(nn.Module):
    def __init__(self, num_classes=4):
        super(MMSER, self).__init__()
        self.num_classes = num_classes
        self.dropout = nn.Dropout(.2)
        # self.linear = nn.Linear(768, 512)
        self.linear1 = nn.Linear(768, 256)
        self.linear2 = nn.Linear(256, 64)
        self.linear3 = nn.Linear(64, num_classes)
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, text_embed, audio_embed):
        # concat_embed = torch.cat((text_embed, audio_embed), dim=1)
        x = self.dropout(text_embed)
        # x = self.linear(x)
        x = self.linear1(x)
        x = self.linear2(x)
        y_logits = self.linear3(x)
        y_softmax = self.softmax(y_logits)
        return y_logits, y_softmax

In [5]:
def calculate_accuracy(y_pred, y_true):
    class_weights = {cls: 1.0/count for cls, count in Counter(y_true).items()}
    wa = balanced_accuracy_score(y_true, y_pred, sample_weight=[class_weights[cls] for cls in y_true])
    ua = accuracy_score(y_true, y_pred)
    return wa, ua

In [6]:
def train_step(model, dataloader, optim, loss_fn, accuracy_fn):
    train_loss = 0.0
    train_wa = 0.0
    train_ua = 0.0
    y_true_ls = []
    y_pred_ls = []
    
    model.train()
    for batch, (text_embed, audio_embed, label) in enumerate(dataloader):
        text_embed = text_embed.to(device)
        audio_embed = audio_embed.to(device)
        label = label.to(device)
        output_logits, output_softmax = model(text_embed, audio_embed)
        output_logits, output_softmax = output_logits.to(device), output_softmax.to(device)
        y_preds = output_softmax.argmax(dim=1).to(device)
        
        wa, ua = calculate_accuracy(y_preds.cpu().numpy(), label.cpu().numpy())
        y_true_ls.append(label.cpu().numpy())
        y_pred_ls.append(y_preds.cpu().numpy())
        loss = loss_fn(output_logits, label)
        
        optim.zero_grad()
        loss.backward()
        optim.step()
        
        train_loss += loss.item()
        train_wa += wa
        train_ua += ua
        
        # if batch % 20 == 0:
        #     print(f"\tBatch {batch}: Train loss: {loss:.5f} | Train WA : {wa:.4f} | Train UA : {ua:.4f}")
        #     print("----------------------------------------")
        
    train_loss /= len(dataloader)
    train_wa /= len(dataloader)
    train_ua /= len(dataloader)
    print(f"Total Train loss: {train_loss:.5f} | Total Train WA : {wa:.4f} | Total Train UA : {ua:.4f}")
    
    return train_loss, train_wa, train_ua, y_true_ls, y_pred_ls

In [7]:
def eval_step(model, dataloader, loss_fn, accuracy_fn):
    eval_loss = 0.0
    eval_wa = 0.0
    eval_ua = 0.0
    y_true_ls = []
    y_pred_ls = []
    
    model.eval()
    with torch.no_grad():
        for batch, (text_embed, audio_embed, label) in enumerate(dataloader):
            text_embed = text_embed.to(device)
            audio_embed = audio_embed.to(device)
            label = label.to(device)
            output_logits, output_softmax = model(text_embed, audio_embed)
            output_logits, output_softmax = output_logits.to(device), output_softmax.to(device)
            y_preds = output_softmax.argmax(dim=1).to(device)
            
            wa, ua = calculate_accuracy(y_preds.cpu().numpy(), label.cpu().numpy())
            y_true_ls.append(label.cpu().numpy())
            y_pred_ls.append(y_preds.cpu().numpy())
            loss = loss_fn(output_logits, label)

            eval_loss += loss.item()
            eval_wa += wa
            eval_ua += ua

            # if batch % 20 == 0:
            #     print(f"\tBatch {batch}: Test loss: {loss:.5f} | Test WA : {wa:.4f} | Test UA : {ua:.4f}")
            #     print("----------------------------------------")
        
        eval_loss /= len(dataloader)
        eval_wa /= len(dataloader)
        eval_ua /= len(dataloader)
        print(f"Total Test loss: {eval_loss:.5f} | Total Test WA: {eval_wa:.4f} | Total Test UA: {eval_ua:4f}")
        
        return eval_loss, eval_wa, eval_ua, y_true_ls, y_pred_ls

In [8]:
epochs = []
train_loss_hist, train_wa_hist, train_ua_hist, val_loss_hist, val_wa_hist, val_ua_hist = [], [], [], [], [], []

best_w = {}

best_train_loss, best_val_loss = 10000, 10000
best_wa, best_ua = 0.0, 0.0

NUM_EPOCHS = 200

model=MMSER(num_classes=4)
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)

criterion = nn.CrossEntropyLoss()
lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)

In [9]:
# Train Multi-modal model
for epoch in range(NUM_EPOCHS):
    print("Epoch", epoch)
    train_loss, train_wa, train_ua, y_true_ls, y_pred_ls = train_step(model, train_dataloader, optimizer, criterion, calculate_accuracy)
    val_loss, val_wa, val_ua, y_true_ls, y_pred_ls = eval_step(model, val_dataloader, criterion, calculate_accuracy)
    lr_scheduler.step()
    
    epochs.append(epoch)
    train_loss_hist.append(train_loss)
    val_loss_hist.append(val_loss)
    train_wa_hist.append(train_wa*100)
    val_wa_hist.append(val_wa*100)
    train_ua_hist.append(train_ua*100)
    val_ua_hist.append(val_ua*100)
    
    if train_loss < best_train_loss and val_loss < best_val_loss:
        best_train_loss, best_val_loss = train_loss, val_loss
        best_wa, best_ua = val_wa, val_ua
        torch.save(model.state_dict(), "C:/Users/admin/Documents/Speech-Emotion_Recognition-2/saved_models/IEMOCAP_ENG_CMN_BERT_wav2vec.pt")
        best_w = model.state_dict()
    
    print("\n==============================\n")
print("Best WA: ", best_wa)
print("Best UA: ", best_ua)


Epoch 0
Total Train loss: 1.16976 | Total Train WA : 0.4470 | Total Train UA : 0.4545
Total Test loss: 1.05404 | Total Test WA: 0.5556 | Total Test UA: 0.556130


Epoch 1
Total Train loss: 1.00127 | Total Train WA : 0.5546 | Total Train UA : 0.5636
Total Test loss: 0.98087 | Total Test WA: 0.5761 | Total Test UA: 0.586699


Epoch 2
Total Train loss: 0.95641 | Total Train WA : 0.5959 | Total Train UA : 0.6182
Total Test loss: 0.96692 | Total Test WA: 0.5764 | Total Test UA: 0.586258


Epoch 3
Total Train loss: 0.93997 | Total Train WA : 0.6236 | Total Train UA : 0.6364
Total Test loss: 0.95610 | Total Test WA: 0.5779 | Total Test UA: 0.589824


Epoch 4
Total Train loss: 0.92561 | Total Train WA : 0.6840 | Total Train UA : 0.6909
Total Test loss: 0.98936 | Total Test WA: 0.5654 | Total Test UA: 0.569511


Epoch 5
Total Train loss: 0.92595 | Total Train WA : 0.6367 | Total Train UA : 0.6364
Total Test loss: 0.95689 | Total Test WA: 0.5962 | Total Test UA: 0.611458


Epoch 6
Total Train lo